In [ ]:
%%writefile submission.py
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import numpy as np
BOARD_LENGTH = 77

# from submission_helpers import *

def locator(locations):
    locs = np.zeros(BOARD_LENGTH)
    for x in locations:
        if type(x) == int:
            locs[x] += 1
        else:
            for spot in x: 
                locs[spot] += 1
    
    return locs

    

def convolve_locator(locator, orientation="horizontal", size = 3):
    ba_list = []
    
    board_array = np.reshape(locator,(7,11))
    
    if orientation == "vertical":
        board_array = board_array.T
        
    for axis_el in board_array:
        ba_list.append(np.convolve(axis_el, np.ones(size), 'same'))
        
    out = np.vstack(ba_list)
    
    if orientation == "vertical":
        return out.T.ravel()
    
    return out.ravel()

def convolve_locator_diagonals(locator):
    pass

def geese_foods_to_vec(observation):
    """
    Binary vector of location of geese/food, convolved
    """
    geese = locator(observation.geese)
    conv_geese = convolve_locator(convolve_locator(geese), orientation = "vertical")
    
    food = locator(observation.food)
    conv_food = convolve_locator(convolve_locator(food), orientation = "vertical")

    return np.hstack((conv_geese, conv_food))
    

def input_vec(observation, action, reward):
    player_index = observation.index
    player_goose = observation.geese[player_index]
    
    return np.hstack((geese_foods_to_vec(observation), locator(player_goose)*player_index, encode_action(action),[reward]))

def encode_action(action_str):
    actions = np.zeros(3)
    if action_str == "SOUTH":
        actions[0] += 1
    elif action_str == "NORTH":
        actions[1] += 1
    elif action_str == "EAST":
        actions[2] += 1
    return actions 

def agent(obs_dict, config_dict):
    """This agent always moves toward observation.food[0] but does not take advantage of board wrapping"""
    observation = Observation(obs_dict)
    vectobs = geese_foods_to_vec(observation)
   
    
    configuration = Configuration(config_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    food = observation.food[0]
    food_row, food_column = row_col(food, configuration.columns)

    input_without_action = np.hstack((vectobs,  locator(player_goose)*player_index))

    
    if food_row > player_row:
        return Action.SOUTH.name
    if food_row < player_row:
        return Action.NORTH.name
    if food_column > player_column:
        return Action.EAST.name
    return Action.WEST.name

In [ ]:
[Link to original NB](https://www.kaggle.com/claurin1/lightgbm-geese-data-generation/output)

In [ ]:
from kaggle_environments import make
import numpy as np
# from submission_helpers import *
import numpy as np
import time
import pandas as pd
import os
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
import numpy as np
BOARD_LENGTH = 77

def locator(locations):
    locs = np.zeros(BOARD_LENGTH)
    for x in locations:
        if type(x) == int:
            locs[x] += 1
        else:
            for spot in x: 
                locs[spot] += 1
    
    return locs

    

def convolve_locator(locator, orientation="horizontal", size = 3):
    ba_list = []
    
    board_array = np.reshape(locator,(7,11))
    
    if orientation == "vertical":
        board_array = board_array.T
        
    for axis_el in board_array:
        ba_list.append(np.convolve(axis_el, np.ones(size), 'same'))
        
    out = np.vstack(ba_list)
    
    if orientation == "vertical":
        return out.T.ravel()
    
    return out.ravel()

def convolve_locator_diagonals(locator):
    pass

def geese_foods_to_vec(observation):
    """
    Binary vector of location of geese/food, convolved
    """
    geese = locator(observation.geese)
    conv_geese = convolve_locator(convolve_locator(geese), orientation = "vertical")
    
    food = locator(observation.food)
    conv_food = convolve_locator(convolve_locator(food), orientation = "vertical")

    return np.hstack((conv_geese, conv_food))
    

def input_vec(observation, action, reward):
    player_index = observation.index
    player_goose = observation.geese[player_index]
    
    return np.hstack((geese_foods_to_vec(observation), locator(player_goose)*player_index, encode_action(action),[reward]))

def encode_action(action_str):
    actions = np.zeros(3)
    if action_str == "SOUTH":
        actions[0] += 1
    elif action_str == "NORTH":
        actions[1] += 1
    elif action_str == "EAST":
        actions[2] += 1
    return actions 

env = make("hungry_geese", debug=True)
out_collect = {}
n_games = 10
for x in range(n_games):
    env.reset()
    env.run(['submission.py', 'submission.py'])
#     env.render(mode="ipython", width=800, height=800)

    out_array = np.vstack([input_vec(step[0].observation, step[0].action, step[0].reward) for step in  env.steps])
    out_columns = [f"goose_pos_{x}" for x in range(77)] + [f"food_pos_{x}" for x in range(77)] + [f"player_pos_{x}" for x in range(77)] + ["North", "South", "East", "Reward"] 
    pa_table = pd.DataFrame(out_array, columns = out_columns)
    out_collect[x] = (pa_table)

    
    
os.makedirs("../output", exist_ok=True)
pa_table_set = pd.concat(out_collect)
pa_table_set = pa_table_set.reset_index().rename(columns = {'level_0':'game', 'level_1':'step'})
pa_table_set.to_csv( f"train_data_{str(int(time.time()))}.csv")

   

In [ ]:
import datetime as dt
str(dt.datetime.now()).rstrip(' ')

In [ ]:
import time
